# TEMP
---


# Import Modules

In [1]:
import os
print(os.getcwd())
import sys

import copy

import numpy as np
import pandas as pd
# pd.set_option("display.max_columns", None)
# pd.set_option('display.max_rows', None)
# pd.options.display.max_colwidth = 100

from sklearn.linear_model import LinearRegression

/mnt/f/Dropbox/01_norskov/00_git_repos/PROJ_IrOx_OER/workflow/oer_vs_features/bivariate_combs_feat


# Script Inputs

In [2]:
# target_ads = "o"
target_ads = "oh"

verbose = True
# verbose = False

# Read Data

In [3]:
from methods import get_df_features_targets
df_features_targets = get_df_features_targets()
df_i = df_features_targets

from methods import get_df_slab
df_slab = get_df_slab()

# Getting phase > 1 slab ids
df_slab_i = df_slab[df_slab.phase > 1]
phase_2_slab_ids = df_slab_i.slab_id.tolist()

### Processing dataframe

In [4]:
if target_ads == "o":
    other_ads = "oh"
elif target_ads == "oh":
    other_ads = "o"

df_i = df_i.drop("features_stan", axis=1, level=0)

df_i = df_i.drop(other_ads, axis=1, level=1)

### Dropping phase 1 slabs

In [5]:
df_index = df_i.index.to_frame()
df_index_i = df_index[
    df_index.slab_id.isin(phase_2_slab_ids)
    ]

print("Dropping phase 1 slabs")
df_i = df_i.loc[
    df_index_i.index
    ]

Dropping phase 1 slabs


In [6]:
df_feat = df_i["features"][target_ads]

### Standardize columns

In [7]:
df_features = df_feat

df_features_stan = copy.deepcopy(df_features)
for col_i in df_features_stan.columns:
    max_val = df_features_stan[col_i].max()
    mean_val = df_features_stan[col_i].mean()
    std_val = df_features_stan[col_i].std()
    # df_features_stan[col_i] = (df_features_stan[col_i] - mean_val) / std_val
    df_features_stan[col_i] = (df_features_stan[col_i]) / max_val
df_feat_stan = df_features_stan

In [8]:
# Create every possible bivariate combination to be tested for feature engineering
from itertools import combinations

column_list = df_feat_stan.columns
interactions = list(combinations(column_list, 2))

interactions

[('active_o_metal_dist', 'eff_oxid_state'),
 ('active_o_metal_dist', 'ir_o_mean'),
 ('active_o_metal_dist', 'ir_o_std'),
 ('active_o_metal_dist', 'octa_vol'),
 ('active_o_metal_dist', 'dH_bulk'),
 ('active_o_metal_dist', 'volume_pa'),
 ('active_o_metal_dist', 'bulk_oxid_state'),
 ('eff_oxid_state', 'ir_o_mean'),
 ('eff_oxid_state', 'ir_o_std'),
 ('eff_oxid_state', 'octa_vol'),
 ('eff_oxid_state', 'dH_bulk'),
 ('eff_oxid_state', 'volume_pa'),
 ('eff_oxid_state', 'bulk_oxid_state'),
 ('ir_o_mean', 'ir_o_std'),
 ('ir_o_mean', 'octa_vol'),
 ('ir_o_mean', 'dH_bulk'),
 ('ir_o_mean', 'volume_pa'),
 ('ir_o_mean', 'bulk_oxid_state'),
 ('ir_o_std', 'octa_vol'),
 ('ir_o_std', 'dH_bulk'),
 ('ir_o_std', 'volume_pa'),
 ('ir_o_std', 'bulk_oxid_state'),
 ('octa_vol', 'dH_bulk'),
 ('octa_vol', 'volume_pa'),
 ('octa_vol', 'bulk_oxid_state'),
 ('dH_bulk', 'volume_pa'),
 ('dH_bulk', 'bulk_oxid_state'),
 ('volume_pa', 'bulk_oxid_state')]

In [9]:
def create_lin_model(
    y=None,
    df_x=None,
    ):
    """
    """
    X_i = df_x_i.to_numpy()
    X_i = X_i.reshape(-1, 1)

    model_i = LinearRegression()
    model_i.fit(X_i, y)

    # #####################################################
    out_dict = dict()
    # #####################################################
    out_dict["model"] = model_i
    out_dict["X"] = X_i
    # #####################################################
    return(out_dict)
    # #####################################################

In [19]:
def local_method_wrap(
    feat_i=None,
    y=None,
    # df_x_i=None,
    df_xy_i=None,
    print_str="",
    ):
    # #####################################################
    # df_x_i = df_xy_i[feat_pair_i[0]]
    df_x_i = df_xy_i[feat_i]
    # #####################################################
    model_dict_i = create_lin_model(y=y, df_x=df_x_i)
    # #####################################################
    model_i = model_dict_i["model"]
    X_i = model_dict_i["X"]
    # #####################################################

    # print("Model 0 Score:", np.round(model_i.score(X_i, y), 3))
    print(print_str + ":", np.round(model_i.score(X_i, y), 3))


for feat_pair_i in interactions:
    print(60 * "=")
    print("feat_pair_i:", feat_pair_i)

    bivar_feat_i = df_feat_stan[feat_pair_i[0]] * df_feat_stan[feat_pair_i[1]]

    feat_name_bivar_i = feat_pair_i[0] + "*" + feat_pair_i[1]
    bivar_feat_i.name = feat_name_bivar_i

    y = df_i["targets"]["g_" + target_ads]

    df_xy_i = pd.concat([
        y,
        bivar_feat_i,
        df_feat_stan[list(feat_pair_i)]
        ], axis=1)
    df_xy_i = df_xy_i.dropna()

    y = df_xy_i["g_" + target_ads]


    df_x_i = df_xy_i[feat_pair_i[0]]
    local_method_wrap(
        feat_i=feat_pair_i[0], y=y, df_xy_i=df_xy_i,
        print_str="Model 0 Score")
    df_x_i = df_xy_i[feat_pair_i[1]]
    local_method_wrap(
        feat_i=feat_pair_i[0], y=y, df_xy_i=df_xy_i,
        print_str="Model 0 Score")
    df_x_i = df_xy_i[feat_name_bivar_i]
    local_method_wrap(
        feat_i=feat_pair_i[0], y=y, df_xy_i=df_xy_i,
        print_str="Model Bivar Score")

    print("")

feat_pair_i: ('active_o_metal_dist', 'eff_oxid_state')
Model 0 Score: 0.148
Model 0 Score: 0.766
Model Bivar Score: 0.776

feat_pair_i: ('active_o_metal_dist', 'ir_o_mean')
Model 0 Score: 0.147
Model 0 Score: 0.699
Model Bivar Score: 0.408

feat_pair_i: ('active_o_metal_dist', 'ir_o_std')
Model 0 Score: 0.147
Model 0 Score: 0.14
Model Bivar Score: 0.148

feat_pair_i: ('active_o_metal_dist', 'octa_vol')
Model 0 Score: 0.107
Model 0 Score: 0.657
Model Bivar Score: 0.591

feat_pair_i: ('active_o_metal_dist', 'dH_bulk')
Model 0 Score: 0.148
Model 0 Score: 0.179
Model Bivar Score: 0.156

feat_pair_i: ('active_o_metal_dist', 'volume_pa')
Model 0 Score: 0.148
Model 0 Score: 0.032
Model Bivar Score: 0.049

feat_pair_i: ('active_o_metal_dist', 'bulk_oxid_state')
Model 0 Score: 0.148
Model 0 Score: 0.502
Model Bivar Score: 0.504

feat_pair_i: ('eff_oxid_state', 'ir_o_mean')
Model 0 Score: 0.768
Model 0 Score: 0.699
Model Bivar Score: 0.745

feat_pair_i: ('eff_oxid_state', 'ir_o_std')
Model 0 Sco

In [ ]:
assert False

In [ ]:

# X_bivar = 


# .to_numpy()

In [ ]:
df_xy_i

In [ ]:
assert False

### Doing same-variable bivariate features

Doesn't seem to do anything for some reason

In [ ]:
for feat_i in df_feat_stan.columns:
    print(60 * "=")
    print("feat_i:", feat_i)

    # ^^^^

    bivar_feat_i = df_feat_stan[feat_i] * df_feat_stan[feat_i]

    feat_name_bivar_i = feat_i + "__bivar"
    bivar_feat_i.name = feat_name_bivar_i

    y = df_i["targets"]["g_" + target_ads]

    df_xy_i = pd.concat([
        y,
        bivar_feat_i,
        df_feat_stan[feat_i],
        ], axis=1)
    df_xy_i = df_xy_i.dropna()


    # #########################################################
    X_reg = df_xy_i[feat_i].to_numpy()
    X_reg = X_reg.reshape(-1, 1)

    y = df_xy_i["g_" + target_ads]


    model_reg = LinearRegression()
    model_reg.fit(X_reg, y)

    y_predict = model_reg.predict(X_reg)

    # print(20 * "-")
    print("Model Regular Score:", model_reg.score(X_reg, y))

    # #########################################################
    X_bivar = df_xy_i[feat_name_bivar_i].to_numpy()
    X_bivar = X_bivar.reshape(-1, 1)

    model_bivar = LinearRegression()
    model_bivar.fit(X_bivar, y)

    y_predict = model_bivar.predict(X_bivar)

    # print(20 * "-")
    print("Model Bivar Score:", model_bivar.score(X_bivar, y))

In [ ]:
# print(
#     "Number of rows in df_features_targets:",
#     df_i.shape[0],
#     )

# # 150

In [ ]:
# # col_i
# # new_col_i
# # col_i + 

# df_features_stan

In [ ]:
# col_i

In [ ]:
# df_i["features_stan"]

In [ ]:
# y

In [ ]:

# df_i["targets"]

In [ ]:
#     y = df_i["targets"]["g_" + target_ads]

# df_xy_i = 
# y = df_xy_i["g_" + target_ads]

In [ ]:
# df_xy_i

In [ ]:
# model_bivar.get_params()

# model_bivar.coef_

In [ ]:
# model_reg.coef_

In [ ]:
# df_xy_i

In [ ]:

# df_feat_stan